# 장르 속성을 이용한 영화 콘텐츠 기반 필터링

## 데이터 로딩 및 가공
- TMDB 5000 데이터 셋 : imdb.com의 영화 중 주요 5000개 영화에 대한 메타 정보를 가공해 Kaggle에서 제공하는 데이터 셋
- https://www.kaggle.com

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./datasets/tmdb_5000_movies.csv')
print(movies.shape)
display(movies.head(2))

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


### 분석에 사용할 주요 컬럼 추출
- id, title, genres, vote_average(평균 평점), vote_count(평점 투표 수), popularity(영화 인기도), keywords, overview(영화 개요)

In [2]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [3]:
print(movies_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB
None


In [4]:
pd.set_option('max_colwidth', None)

In [5]:
display(movies_df[['genres']])

,genres
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]"
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]"
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]"
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name"": ""Thriller""}]"
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fiction""}]"
...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 53, ""name"": ""Thriller""}]"
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""name"": ""Romance""}]"
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""name"": ""Romance""}, {""id"": 10770, ""name"": ""TV Movie""}]"
4801,[]


In [6]:
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

- eval() 함수를 통해 List 객체로 변환
>- eval() : 문자열 형태로 되어 있는 표현식을 실행하여 파이썬 코드로 변환하는 함수

In [7]:
movies_df['genres'] = movies_df['genres'].apply(eval)
movies_df['keywords'] = movies_df['keywords'].apply(eval)

- 장르와 키워드 컬럼의 name 키의 값만 원소로 출력하여 리스트로 생성

In [8]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[ y['name'] for y in x ]) # x : 리스트 객체, y : 딕셔너리 값 
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[ y['name'] for y in x ])

## 장르 콘텐츠 유사도 측정
- 문자열로 변환된 genres 컬럼은 카운트 기반으로 피처 벡터화 변환
- genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교
- 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화를 추천

In [9]:
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x:' '.join(x))

In [10]:
movies_df['genres_literal'][0]

'Action Adventure Fantasy Science Fiction'

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])

In [12]:
print(genre_mat.shape)
print(genre_mat.toarray()[0])

(4803, 276)
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[0])
print(genre_sim[1])
print(genre_sim[2])

(4803, 4803)
[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
[0.59628479 1.         0.4        ... 0.         0.         0.        ]
[0.4472136 0.4       1.        ... 0.        0.        0.       ]


In [14]:
genre_sim_sorted_idx = genre_sim.argsort(axis = 1)[:,::-1]
print(genre_sim_sorted_idx[0,:10])

[   0 3494  813  870   46   14 1296 1652  419  420]


In [15]:
# 첫번째 영화와 장르 유사도가 가장 높은 3494번째 영화의 유사도 값 확인
print(genre_sim[0, 3494])

1.0000000000000002


In [16]:
print(movies_df.iloc[0,:]['title'])
print(movies_df.iloc[3494,:]['title'])

Avatar
Beastmaster 2: Through the Portal of Time


In [34]:
movies_df[movies_df['title']=='The Godfather']

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
3337,238,The Godfather,"[Drama, Crime]",8.4,5893,143.659698,"[italy, love at first sight, loss of father, patriarch, organized crime, mafia, lawyer, italian american, crime family, rise to power, mob boss, 1940s]","Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime family. When organized crime family patriarch, Vito Corleone barely survives an attempt on his life, his youngest son, Michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.",Drama Crime


In [27]:
# df: 영화 정보를 갖는 데이터 프레임
# sorted_idx : 장르별 유사도가 높은 순으로 정렬된 영화 인덱스 행렬
# title_name : 추천 기준이 되는 영화 제목
# top_n : 추천 영화 수
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    target_movie = df[df['title']==title_name]
    title_index = target_movie.index.values # index : 인덱스 값, values : 넘파이의 다차원 배열로 가져옴
    similar_indexes = sorted_idx[title_index, :top_n]
    similar_indexes = similar_indexes.reshape(-1) # 2차원 벡터를 1차원 벡터의 형태로 만듦

    return df.iloc[similar_indexes] # iloc : 인덱스 값으로 접근

- 평점이 매우 낮은 영화도 추천 영화로 선택되는 문제가 발생
- 좀 더 많은 후보군을 선정한 뒤 영화의 평점에 따라 필터링하는 방식으로 변경 필요

In [28]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_idx, 'The Godfather', 10)
display(similar_movies[['title','vote_average']])

,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- 영화 평점이 높은 순으로 정렬하여 상위 10위 영화 확인

In [29]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 왜곡된 평점 데이터를 피하기 위해 평점에 `평가 횟수를 반영한 가중 평점 방식` 적용 (IMDB에서 평점을 매기는 방식)
- 가중 평점 (Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
>- v : 개별 영화에 평점을 투표한 횟수
>- m : 평점을 부여하기 위한 최소 투표 횟수 (임의의 값<전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 적용>)
>- R : 개별 영화에 대한 평균 평점(vote_average 값)
>- C : 전체 영화에 대한 평균 평점(vote_average 평균 값)

In [34]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(f'C:{C:.3f}, m:{m:.3f}')

C:6.092, m:370.200


In [35]:
def weighted_vote_average(data): # v와 R값만
    v = data['vote_count']
    R = data['vote_average']
    return ((v/(v+m)) * R) + ((m/(v+m)) * C)

In [36]:
movies_df['weighted_rating'] = movies_df.apply(weighted_vote_average, axis = 1)

In [37]:
movies_df[['title','vote_average','weighted_rating','vote_count']].sort_values('weighted_rating', ascending=False)[:10]

,title,vote_average,weighted_rating,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [38]:
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    target_movie = df[df['title']==title_name]
    title_index = target_movie.index.values 
    
    # top_n의 2배에 해당하는 장르 유사도가 높은 영화 index 추출 
    similar_indexes = sorted_idx[title_index, :top_n*2]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_rating', ascending=False)[:top_n]

In [39]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_idx, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_rating']]

,title,vote_average,weighted_rating
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
